In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

/var/folders/6d/m4g0vvg508z0xb6_c6h4qmfm0000gn/T/ipykernel_2996/2721483381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
sp500_futures = yf.download('ES=F')

/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Log returns
sp500_futures['Log Returns'] = np.log(sp500_futures['Adj Close']/sp500_futures['Adj Close'].shift(1))
# Volatility
volatility = sp500_futures['Log Returns'].std()
#volatility = sum((np.exp(sp500_futures['Log Returns']) - np.exp(sp500_futures['Log Returns']).mean)**2)/(len(sp500_futures['Log Returns'])-1)

In [4]:
#Annualized returns and volatility
annualized_return = sp500_futures['Log Returns'].mean() * np.sqrt(252)
annualized_volatility = volatility * np.sqrt(252)

In [16]:
sp500_futures.index[0]
sp500_futures.index[-1]
#sp500_start_date = str(sp500_futures.index[0])[:10]
#risk_free_rate_series = yf.download('^IRX', start = sp500_start_date)['Adj Close']
#risk_free_rate_series

Timestamp('2024-02-02 00:00:00')

In [6]:
# Downloading Risk-Free Rate Series (3-month Treasury bill)
sp500_start_date = str(sp500_futures.index[0])[:10]
risk_free_rate_series = yf.download('^IRX', start = sp500_start_date)['Adj Close']
risk_free_rate = risk_free_rate_series.mean() / 100

/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [7]:
sharpe_ratio = (annualized_return - risk_free_rate)/annualized_volatility
print("annualized_return:", annualized_return)
print("risk_free_rate_annual:", risk_free_rate)
print("annualized_volatility:", annualized_volatility)
print("sharpe_ratio:", sharpe_ratio)

annualized_return: 0.003280855470044652
risk_free_rate_annual: 0.015436050387678568
annualized_volatility: 0.19655056187268022
sharpe_ratio: -0.06184258544886634


In [8]:
# sharpe ratio for the 10-year Treasury futures (ZN=F)

In [9]:
# doawnloading futures
treasury_futures = yf.download('ZN=F')

# Log returns
treasury_futures['Log Returns'] = np.log(treasury_futures['Adj Close']/treasury_futures['Adj Close'].shift(1))

# Volatility
volatility = treasury_futures['Log Returns'].std()

#Annualized returns and volatility
annualized_return = treasury_futures['Log Returns'].mean() * np.sqrt(252) * np.sqrt(10)
annualized_volatility = volatility * np.sqrt(252) * np.sqrt(10)

# Downloading Risk-Free Rate Series (3-month Treasury bill)
treasury_start_date = str(sp500_futures.index[0])[:10]
risk_free_rate_series = yf.download('^IRX', start = treasury_start_date)['Adj Close']
risk_free_rate = risk_free_rate_series.mean() / 100

sharpe_ratio = (annualized_return - risk_free_rate)/annualized_volatility
print("annualized_return:", annualized_return)
print("risk_free_rate_annual:", risk_free_rate)
print("annualized_volatility:", annualized_volatility)
print("sharpe_ratio:", sharpe_ratio) # greater in absolute value than sp500 => correct

/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

annualized_return: 0.0009533646193404857
risk_free_rate_annual: 0.015436050387678568
annualized_volatility: 0.19672608861670648
sharpe_ratio: -0.07361853158457081


In [10]:
# mixed portfolio with 60% weight in S&P 500 futures (ES=F) and 40% weight in 10-year Treasury futures (ZN=F)
# getting futures
sp500_futures = yf.download('ES=F')['Adj Close']
treasury_futures = yf.download('ZN=F')['Adj Close']
data = pd.DataFrame({'treasury_futures': treasury_futures , 'sp500_futures': sp500_futures}).dropna()

# Log returns
data['Log Returns'] = np.log(data['treasury_futures']/data['treasury_futures'].shift(1)) * 0.4 + np.log(data['sp500_futures']/data['sp500_futures'].shift(1)) * 0.6

# Volatility
volatility = data['Log Returns'].std()

#Annualized returns and volatility
annualized_return = data['Log Returns'].mean() * np.sqrt(252) * np.sqrt(10)
annualized_volatility = volatility * np.sqrt(252) * np.sqrt(10)

# Downloading Risk-Free Rate Series (3-month Treasury bill)
start_date = '2000-09-18'
risk_free_rate_series = yf.download('^IRX', start = start_date)['Adj Close']
risk_free_rate = risk_free_rate_series.mean() / 100

sharpe_ratio = (annualized_return - risk_free_rate)/annualized_volatility
print("annualized_return:", annualized_return)
print("risk_free_rate_annual:", risk_free_rate)
print("annualized_volatility:", annualized_volatility)
print("sharpe_ratio:", sharpe_ratio) # greater than sp500 in absolute value => correct


/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************

annualized_return: 0.006644188091548202
risk_free_rate_annual: 0.015436050387678568
annualized_volatility: 0.3609127058929428
sharpe_ratio: -0.024360079743876595


In [20]:
# INFLATION ADJUSTED SHARPE RATIO
# mixed portfolio with 60% weight in S&P 500 futures (ES=F) and 40% weight in 10-year Treasury futures (ZN=F)
# getting futures

import pandas_datareader as pdr

sp500_futures = yf.download('ES=F')['Adj Close']
treasury_futures = yf.download('ZN=F')['Adj Close']
data = pd.DataFrame({'treasury_futures': treasury_futures , 'sp500_futures': sp500_futures}).dropna()

# Log returns
data['Log Returns'] = np.log(data['treasury_futures']/data['treasury_futures'].shift(1)) * 0.4 + np.log(data['sp500_futures']/data['sp500_futures'].shift(1)) * 0.6

# Volatility
volatility = data['Log Returns'].std()

#Annualized returns and volatility
annualized_return = data['Log Returns'].mean() * np.sqrt(252) * np.sqrt(10)
annualized_volatility = volatility * np.sqrt(252) * np.sqrt(10)

# data source: 'fred' (Federal Reserve Economic Data)
data_source = 'fred'

# series ID: Consumer Price Index for All Urban Consumers (CPIAUCSL)
series_id = 'CPIAUCSL'

# start and end dates for the data
start_date = '2000-09-18'
end_date = '2024-02-02'

# fetch inflation data
inflation_data = pdr.get_data_fred(series_id, start_date, end_date)

# compute annualized inflation rate
inflation_rate = inflation_data['CPIAUCSL'].pct_change(periods=12)

# resample monthly inflation data to daily frequency (forward fill)
inflation_rate = inflation_rate.resample('D').ffill

sharpe_ratio = (annualized_return - inflation_rate/annualized_volatility
print("annualized_return:", annualized_return)
print("risk_free_rate_annual:", risk_free_rate)
print("annualized_volatility:", annualized_volatility)
print("sharpe_ratio:", sharpe_ratio) # greater than sp500 in absolute value => correct

/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/nossa/miniconda3/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


AttributeError: 'function' object has no attribute 'mean'